# Query 1
user: How many artists feature Santana?


## Answer 1 
- incorrect, queried albums
assistant:
{'name': 'ask_database', 'arguments': '{\n  "query": "SELECT COUNT(DISTINCT ArtistId) FROM albums WHERE Title LIKE \'%Santana%\'"\n}'}


## Answer 2
- correct, gave 9 results
assistant:
assistant: {'name': 'ask_database', 'arguments': '{\n  "query":"SELECT COUNT(*) FROM artists WHERE Name LIKE \'%Santana%\'"\n}'}

# Query 2
user: how many artists are there? Correct

In [ ]:
system: Answer user questions by generating SQL queries against the Chinook Music Database.
assistant: The genre with the least tracks in the Chinook Music Database is Opera. It only has 1 track.

# Query 3
What track is the most purchased?

Results: 
Not incorrect but there are many tracks that have been purchased twice. Using limit 1 ignores ties

Better query is sth like

`select t.Name from tracks t inner join
(select TrackId, count(*) c  from invoice_items group by TrackId  order by c desc)q
on t.TrackId = q.TrackId
where q.c = (select max(c) from (select TrackId, count(*) c  from invoice_items group by TrackId  order by c desc)q2)`

which  gets all the tied max download

Changign the prompt to ' "What tracks are the most purchased"' gets it to return the 5 highest, still using limit.

In [ ]:
system: Answer user questions by generating SQL queries against the Chinook Music Database.

user: What track is the most purchased?

assistant: {'name': 'ask_database', 'arguments': '{\n  "query": "SELECT tracks.Name, count(*) as purchased_times FROM invoice_items JOIN tracks ON tracks.TrackId = invoice_items.TrackId GROUP BY tracks.TrackId ORDER BY purchased_times DESC LIMIT 1"\n}'}

function (ask_database): [('Balls to the Wall', 2)]

assistant: The most purchased track is "Balls to the Wall" with 2 purchases.

## List the 5 highest spending customers?

Nailed it. Got the correct result with a shorter query than what I came up with. It noticed there is already a column.
It assumed costs were in $ but it is not known what is the currency.
for the index total.
This was my query:

```
select i.CustomerId, c.FirstName || ' ' ||  c.LastName as customerName,  round(sum(spendPerInvoice.totalSpent),2) as spendPerCustomer 
from invoices i
inner join
(select InvoiceId, round(sum(UnitPrice * Quantity),2) as totalSpent 
  from invoice_items
  group by InvoiceId 
  order by totalSpent desc) spendPerInvoice
on i.InvoiceId = spendPerInvoice.InvoiceId
inner join customers c
on c.CustomerId=i.CustomerId
group by i.CustomerId
order by spendPerCustomer desc
limit 5
```

In [ ]:
user: List the 5 highest spending customers?

assistant: {'name': 'ask_database', 'arguments': 
            '{\n  "query": "SELECT customers.FirstName, customers.LastName, SUM(invoices.Total) as TotalSpent FROM customers INNER JOIN invoices ON customers.CustomerId = invoices.CustomerId GROUP BY customers.CustomerId ORDER BY TotalSpent DESC LIMIT 5"\n}'}

function (ask_database): [('Helena', 'Holý', 49.620000000000005), ('Richard', 'Cunningham', 47.620000000000005), ('Luis', 'Rojas', 46.62), ('Ladislav', 'Kovács', 45.62), ('Hugh', "O'Reilly", 45.62)]

assistant: The 5 highest spending customers are:

1. Helena Holý: $49.62
2. Richard Cunningham: $47.62
3. Luis Rojas: $46.62
4. Ladislav Kovács: $45.62
5. Hugh O'Reilly: $45.62